---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE TICKER

In [1]:
ticker = "IBM"

# GET DATA

In [2]:
import yfinance as yf
from pandas_datareader import DataReader as pdr

 # get monthly market excess return and risk-free rate for past 60 months from French's data library
ff = pdr('F-F_Research_Data_Factors','famafrench',start=1970)[0] / 100
ff = ff.iloc[-60:]

# pull data from yahoo for ticker
ticker = ticker.upper()
ret = yf.download(ticker, start='2017-01-01', progress=False)['Adj Close']
ret = ret.resample('M').last().pct_change()
ret.index = ret.index.to_period('M')
ret.name = "ret"

# Combine with market data
df = ff.join(ret, how="left")
df["ret"] -= df.RF
df = df[["Mkt-RF", "ret"]].reset_index()
df.columns = ["date", "mkt", "ret"]
df["date"] = df.date.astype(str)

# RUN MARKET MODEL REGRESSION

In [6]:
import statsmodels.formula.api as smf

result = smf.ols("ret ~ mkt", data=df).fit()
alpha = result.params["Intercept"]*100*100
beta  = result.params["mkt"]
print(f'{ticker} Alpha (bps/month):\t{alpha:.2f}')
print(f'{ticker} Beta:\t\t{beta:.2f}')

IBM Alpha (bps/month):	-10.76
IBM Beta:		0.80


# FIGURE

In [7]:
import plotly.express as px

fig = px.scatter(
    df,
    x="mkt",
    y="ret",
    trendline="ols",
    hover_data=dict(ret=False, mkt=False, date=False),
    hover_name="date",
)

fig.update_traces(
    marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)

fig.update_layout(
    xaxis_title="Market Excess Return",
    yaxis_title= ticker.upper() + " Excess Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
    showlegend=False
)

fig.show()